In [59]:
import pandas as pd
from Bio.SeqUtils import seq1
import re
import requests
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import os
import plotly.graph_objects as go

# import from ../utils.py
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import get_uniprot_sequence, read_finalsum, read_finalsum_decomp, align_finalsum_with_uniprot


from scipy.stats import spearmanr

In [60]:
def check_sequence(mut_df, uniprot_id, aa_col, score_col, convert_to_seq1=True):
    # Check the sequence of the pdb structure against the uniprot sequence
    # aa_col should have either the format <original_3letterAA><residueNum><mutated_3letterAA> or the 1 letter code (if convert_to_seq1 is False)
    # returns a dataframe with columns: mutagenesis_aa, res_num, mutagenesis_functional_score
    
    sequence = get_uniprot_sequence(uniprot_id)
    if sequence is None:
        print("Failed to fetch sequence")
        return None

    # Define regular expressions to extract components
    pattern = re.compile(r'([A-Za-z]+)(\d+)([A-Za-z]+)')  # has format like 'R248W'

    # Iterate over mutations and extract components
    tmp = list()
    for _, row in mut_df.iterrows():
        mutation = str(row[aa_col])
        # if mutation is not none or na and does not have '=' in it
        if mutation and '=' not in mutation and 'nan' not in mutation:
            # mutation = mutation.split('.')[1]
            try:
                original_aa, residue_number, _ = pattern.match(mutation).groups() # extract original amino acid and residue number
            except:
                print("Error in row:", mutation)
                continue
            if convert_to_seq1:
                original_aa = seq1(original_aa)

            tmp.append((original_aa, int(residue_number), row[score_col]))
    
    # unique values in tmp ordered by residue number
    tmp = sorted(list(set(tmp)), key=lambda x: x[1])
    tmp = pd.DataFrame(tmp, columns=['mutagenesis_aa', 'res_num', 'mutagenesis_functional_score'])
        
    
    # check if the sequence is correct
    for _, row in tmp.iterrows():
        if row['res_num'] > len(sequence):
            print(f"Residue number {row['res_num']} exceeds sequence length. Stopping...")
            break
        else:
            if not sequence[row['res_num'] - 1] == row['mutagenesis_aa']:
                print("Mismatch at", row['res_num'], "| Expected:", row['mutagenesis_aa'], "| Found:", sequence[row['res_num'] - 1])
                continue
    
    return tmp

In [61]:
all_mut_df = pd.DataFrame()

## TP53

In [62]:
gene = "TP53"
uniprot_id = "P04637"

In [63]:
# mut_df = pd.read_csv('data/urn_mavedb_00000068-a-1_scores.csv')
# mut_df['aa'] = mut_df['hgvs_pro'].apply(lambda x: x.split('.')[1])
# mut_df = check_sequence(mut_df, uniprot_id, 'aa', 'score', convert_to_seq1=True)
# mut_df = mut_df.groupby('res_num')['mutagenesis_functional_score'].mean().reset_index() # group by residue number and amino acid and take the mean of the scores
# mut_df['gene'] = gene
# mut_df.head()

mut_df = pd.read_excel('data/mmc4.xlsx')
mut_df.head()

,Codon_num,AA_change,RFS_H1299,IARC_occurrences
0,175,R>H,0.239839,1221
1,248,R>Q,0.108320,955
2,273,R>H,0.392768,863
3,248,R>W,0.050891,770
4,273,R>C,-0.127543,718


In [64]:
mut_df['AA_change'].unique()

array(['R>H', 'R>Q', 'R>W', 'R>C', 'G>S', 'R>S', 'Y>C', 'R>*', 'V>F',
       'C>F', 'M>I', 'E>K', 'H>R', 'G>D', 'R>L', 'H>Y', 'S>F', 'V>M',
       'W>*', 'Q>*', 'R>T', 'I>T', 'P>S', 'C>Y', 'V>L', 'G>C', 'G>E',
       'P>L', 'G>V', 'A>T', 'G>R', 'R>K', 'K>N', 'R>F.S', 'R>M', 'L>R',
       'K>R', 'H>L', 'P>F.S', 'M>V', 'A>V', 'R>G', 'D>E', 'N>D', 'E>*',
       'S>F.S', 'R>P', 'P>R', 'P>H', 'D>H', 'I>F', 'S>*', 'C>F.S', 'D>Y',
       'D>N', 'C>S', 'N>S', 'S>C', 'P>T', 'T>N', 'N>F.S', 'A>P', 'Y>H',
       'F>L', 'S>R', 'Y>*', 'F>F.S', 'H>Q', 'P>A', 'L>F', 'G>*', 'C>W',
       'S>I', 'E>F.S', 'F>C', 'T>I', 'I>S', 'R>I', 'H>F.S', 'K>E', 'Y>N',
       'H>N', 'K>*', 'H>D', 'C>R', 'L>P', 'L>V', 'T>P', 'G>F.S', 'S>Y',
       'A>D', 'M>K', 'E>G', 'R>R', 'A>F.S', 'V>G', 'C>*', 'A>S', 'V>A',
       'I>N', 'E>V', 'S>G', 'D>V', 'L>Q', 'Y>S', 'I>F.S', 'E>D', 'G>G',
       'P>P', 'S>N', 'V>I', 'K>Q', 'Y>D', 'H>P', 'A>A', 'V>F.S', 'F>S',
       'D>G', 'K>F.S', 'T>T', 'N>N', 'E>Q', 'D>F.S', 'V>V', 'V>D',

In [65]:
conseq = []
for change in mut_df['AA_change']:
    if "*" in change:
        conseq.append("deletion")
    elif "F.S" in change:
        conseq.append("frameshift")
    else:
        conseq.append("missense")
mut_df['consequence'] = conseq

In [66]:
# from paper:
#  the RFS of all single amino acid substitutions (one letter codes on the left), deletions (“0”), premature stop codons (“∗”), and frameshifts (“F.S.”) are presented

# # we will only consider single amino acid substitutions
# mut_df = mut_df[~mut_df['AA_change'].str.contains(r'0|\*|F\.S\.', na=False)]
mut_df = mut_df.groupby("Codon_num")['RFS_H1299'].mean().reset_index()
mut_df.columns = ['res_num', 'mutagenesis_functional_score']
mut_df['gene'] = gene

In [67]:
all_mut_df = pd.concat([all_mut_df, mut_df])

# read NS data from lung genes

In [83]:
ns_data = pd.read_csv("../lung_cancer/wt_ns_aggregated.csv")
ns_data = ns_data[ns_data['gene'].isin(['TP53', 'EGFR', 'KRAS'])]
print(len(ns_data))
ns_data.head()

689


,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,cbioportal_freq,uniprot_aa_type
71,EGFR,766,M,M,3.057263,0.607900,1.585044,-0.864319,1,oncogene,0,Nonpolar
72,EGFR,793,M,M,4.957605,1.445420,2.301859,-1.210326,1,oncogene,0,Nonpolar
73,EGFR,825,M,M,2.652895,1.789380,0.411729,-0.451786,1,oncogene,0,Nonpolar
74,EGFR,881,M,M,0.908402,0.663610,-0.032369,-0.277160,1,oncogene,0,Nonpolar
75,EGFR,908,M,M,0.891397,1.190479,-0.233699,0.065383,1,oncogene,0,Nonpolar


In [84]:
all_mut_df

,res_num,mutagenesis_functional_score,gene
0,102,-1.568379,TP53
1,103,-0.628484,TP53
2,104,-0.745010,TP53
3,105,-0.835696,TP53
4,106,-1.597091,TP53
...,...,...,...
187,289,-1.774106,TP53
188,290,-1.237663,TP53
189,291,-1.317062,TP53
190,292,-1.707734,TP53


In [85]:
ns_data = ns_data.merge(all_mut_df, left_on=['gene', 'uniprot_num'], right_on=['gene', 'res_num'], how='inner')
ns_data.head()

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,cbioportal_freq,uniprot_aa_type,res_num,mutagenesis_functional_score
0,TP53,133,M,M,1.191576,1.197680,-0.006104,0.0,1,TSG,8,Nonpolar,133,-1.073129
1,TP53,160,M,M,1.077498,0.054685,1.022812,0.0,1,TSG,4,Nonpolar,160,-1.710979
2,TP53,169,M,M,-0.607553,-0.568627,-0.038926,0.0,1,TSG,2,Nonpolar,169,-2.041208
3,TP53,237,M,M,1.469696,0.473786,0.995910,0.0,1,TSG,81,Nonpolar,237,-0.756014
4,TP53,243,M,M,-0.776035,-0.527102,-0.248933,0.0,1,TSG,0,Nonpolar,243,-1.709239


In [86]:
titles = {"TP53": "", "EGFR": "EGFR", "KRAS": "KRAS"}
scores = {"TP53": "Relative Fitness Score (RFS)", "EGFR": "Enrichment Score", "KRAS": "Folding Free Energy Change"}
texts = {"TP53": "The RFS of a mutant amino acid indicates its relative ability to retain <br>the wild-type functionality. Less negative RFS was related to ",
         "EGFR": "The enrichment score at a mutant amino acid here measures how overrepresented<br> or underrepresented compared to the wild type in response to erlotinib treatment",
         "KRAS": "The folding free energy change here represents how much mutation at that residue <br>affects the stability of protein folding"}

for gene in ['TP53']:#, 'EGFR', 'KRAS']:
    tmp = ns_data[ns_data['gene']==gene].reset_index(drop=True)
    corr, p_value = spearmanr(tmp['network_score'], tmp['mutagenesis_functional_score'])
    print(f"Correlation between network score and functional score for {gene}: {corr} (p-value: {p_value})")
    # plot network score vs functional score using plotly
    # Create scatter plot
    fig = go.Figure()
    # figure size
    fig.update_layout(
        autosize=False,
        width=800,
        height=600,
    )
    # color by 
    fig.add_trace(go.Scatter(x=tmp['network_score'], y=tmp['mutagenesis_functional_score'],
                            mode='markers', marker=dict(color='blue'),
                            name=f'Network Score vs. {scores[gene]} per residue', text=tmp['res_num']))

    # Update layout
    fig.update_layout(xaxis_title='Network Score', yaxis_title=scores[gene],
                    title=f'{gene}: Network Score vs. {scores[gene]} per residue',)
    # add small text at the bottom
    fig.add_annotation(text=texts[gene],
                    xref="paper", yref="paper",
                    x=0.5, y=-0.2, showarrow=False, )
    
    if gene == "TP53":
        # color residue number 217, 157 and 257
        # get the index of the residues
        tmp2 = tmp[tmp['uniprot_num'].isin([175, 245, 249, 282])].reset_index(drop=True)
        fig.add_trace(go.Scatter(x=tmp2['network_score'], y=tmp2['mutagenesis_functional_score'],
                            mode='markers', marker=dict(color='red'),
                            name='Residues highlighted', text=tmp2['res_num']))
    

    # make opacity 0.4
    fig.update_traces(marker=dict(size=5, opacity=0.4))

    fig.show()

Correlation between network score and functional score for TP53: 0.5461356575480266 (p-value: 5.226046736410789e-16)


In [20]:
# ns_data[(ns_data['gene']=='EGFR') & (ns_data['mutagenesis_functional_score'] > 0.5)]
ns_data[(ns_data['gene']=='EGFR') & (ns_data['uniprot_num'] == 790)]
# ns_data[(ns_data['gene']=='TP53') & (ns_data['uniprot_num'] == 175)]
# ns_data[(ns_data['gene']=='KRAS') & (ns_data['uniprot_num'] == 13)]

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,res_num,mutagenesis_functional_score
93,EGFR,790,T,T,2.706997,1.275217,0.255643,-1.176137,1,oncogene,790,1.044958


# plot difference in network scores vs functional score

In [70]:
ns_data = pd.read_csv('../lung_cancer/lung_genes_sbna.csv')
ns_data = ns_data[ns_data['uniprot_num']!="?"]
ns_data['uniprot_num'] = ns_data['uniprot_num'].astype(int)
ns_data.head()

,gene,pdb_id,chain,sbna_aa3,sbna_num,sbna_aa1,uniprot_num,uniprot_aa,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,mutant,mutations,cgc_tier,cancer_role,uniprot_aa_type,sbna_aa_type
0,AKT1,2UVM,A,MET,1,M,1,M,-3.289764,-0.887739,-0.599135,1.802890,N,NaN,1,oncogene,Nonpolar,Nonpolar
1,AKT1,2UVM,A,SER,2,S,2,S,-3.211822,-0.780412,-0.514186,1.917224,N,NaN,1,oncogene,Polar,Polar
2,AKT1,2UVM,A,ASP,3,D,3,D,-2.323640,-0.348912,-0.599135,1.375593,N,NaN,1,oncogene,Charged,Charged
3,AKT1,2UVM,A,VAL,4,V,4,V,-1.720478,-0.423430,-0.430810,0.866237,N,NaN,1,oncogene,Nonpolar,Nonpolar
4,AKT1,2UVM,A,ALA,5,A,5,A,-0.735485,-0.359177,0.880892,1.257200,N,NaN,1,oncogene,Nonpolar,Nonpolar


In [71]:
ns_data = ns_data[ns_data['pdb_id'].isin(['2OCJ', '4IBQ', '4IBS'])]

In [72]:
# Create separate DataFrames for each pdb_id
df_2OCJ = ns_data[ns_data['pdb_id'] == '2OCJ'].set_index(['chain', 'uniprot_num'])
df_4IBS = ns_data[ns_data['pdb_id'] == '4IBS'].set_index(['chain', 'uniprot_num'])
df_4IBQ = ns_data[ns_data['pdb_id'] == '4IBQ'].set_index(['chain', 'uniprot_num'])

# Merge DataFrames on chain and uniprot_num
merged_4IBS = df_4IBS.join(df_2OCJ, lsuffix='_4IBS', rsuffix='_2OCJ', how='inner')
merged_4IBQ = df_4IBQ.join(df_2OCJ, lsuffix='_4IBQ', rsuffix='_2OCJ', how='inner')

# Calculate differences in network scores
merged_4IBS['score_diff_4IBS_2OCJ'] = merged_4IBS['network_score_4IBS'] - merged_4IBS['network_score_2OCJ']
merged_4IBQ['score_diff_4IBQ_2OCJ'] = merged_4IBQ['network_score_4IBQ'] - merged_4IBQ['network_score_2OCJ']

# Reset index to make it look cleaner
result_4IBS = merged_4IBS.reset_index()[['chain', 'uniprot_num', 'score_diff_4IBS_2OCJ']]
result_4IBQ = merged_4IBQ.reset_index()[['chain', 'uniprot_num', 'score_diff_4IBQ_2OCJ']]

result_4IBQ

,chain,uniprot_num,score_diff_4IBQ_2OCJ
0,A,94,-0.154586
1,A,97,0.330575
2,A,98,1.357489
3,A,99,0.017505
4,A,100,-0.329625
...,...,...,...
767,D,284,1.684829
768,D,285,0.458237
769,D,286,1.629800
770,D,287,2.014488


In [74]:
result_4IBS.merge(all_mut_df, left_on=['uniprot_num'], right_on=['res_num'], how='inner')

,chain,uniprot_num,score_diff_4IBS_2OCJ,res_num,mutagenesis_functional_score,gene
0,A,102,2.206789,102,-1.568379,TP53
1,B,102,2.206789,102,-1.568379,TP53
2,C,102,2.206789,102,-1.568379,TP53
3,D,102,2.206789,102,-1.568379,TP53
4,A,103,1.948168,103,-0.628484,TP53
...,...,...,...,...,...,...
747,D,288,1.165384,288,-1.409750,TP53
748,A,289,1.971245,289,-1.774106,TP53
749,B,289,1.971245,289,-1.774106,TP53
750,C,289,1.971245,289,-1.774106,TP53


In [80]:
# plot network score difference vs functional score using plotly\

tmp = result_4IBQ.merge(all_mut_df, left_on=['uniprot_num'], right_on=['res_num'], how='inner')
tmp = tmp[tmp['gene']==gene].reset_index(drop=True)
corr, p_value = spearmanr(tmp['score_diff_4IBQ_2OCJ'], tmp['mutagenesis_functional_score'])
print(f"Correlation between network score difference and functional score for {gene}: {corr} (p-value: {p_value})")
# plot network score vs functional score using plotly
# Create scatter plot
fig = go.Figure()
# figure size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)
# color by 
fig.add_trace(go.Scatter
                (x=tmp['score_diff_4IBQ_2OCJ'], y=tmp['mutagenesis_functional_score'],
                            mode='markers', marker=dict(color='blue'),
                            name=f'Network Score Difference vs. {scores[gene]} per residue', text=tmp['res_num']))

# Update layout
fig.update_layout(xaxis_title='Network Score Difference', yaxis_title=scores[gene],
                title=f'TP53: Differences in network scores vs RFS for 4IBQ (R273H mutation)',)
# make opacity 0.4
fig.update_traces(marker=dict(size=5, opacity=0.4))

fig.show()

Correlation between network score difference and functional score for TP53: -0.20616275208205226 (p-value: 1.2683558635491583e-08)


In [81]:
# plot network score difference vs functional score using plotly\

tmp = result_4IBS.merge(all_mut_df, left_on=['uniprot_num'], right_on=['res_num'], how='inner')
tmp = tmp[tmp['gene']==gene].reset_index(drop=True)
corr, p_value = spearmanr(tmp['score_diff_4IBS_2OCJ'], tmp['mutagenesis_functional_score'])
print(f"Correlation between network score difference and functional score for {gene}: {corr} (p-value: {p_value})")
# plot network score vs functional score using plotly
# Create scatter plot
fig = go.Figure()
# figure size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)
# color by 
fig.add_trace(go.Scatter
                (x=tmp['score_diff_4IBS_2OCJ'], y=tmp['mutagenesis_functional_score'],
                            mode='markers', marker=dict(color='blue'),
                            name=f'Network Score Difference vs. {scores[gene]} per residue', text=tmp['res_num']))

# Update layout
fig.update_layout(xaxis_title='Network Score Difference', yaxis_title=scores[gene],
                title=f'TP53: Differences in network scores vs RFS for 4IBS (R273C mutation)',)
# make opacity 0.4
fig.update_traces(marker=dict(size=5, opacity=0.4))

fig.show()

Correlation between network score difference and functional score for TP53: -0.13783335682082382 (p-value: 0.0001496950191622868)


### 1. save to pdf for each pdb

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

for gene in ['TP53']:

    # Initialize a PdfPages object to save plots in a PDF file
    pdf_pages = PdfPages(f'results/{gene}_scatterplots.pdf')

    # plot subplots with 3 columns showing the network score against mutagenesis_functional_score for each pdb_chain in seaborn
    unique_pdb_chains = residue_data[residue_data['assoc_gene']==gene]['pdb_chain'].unique()
    n = len(unique_pdb_chains)
    rows = n // 3 + 1

    fig, axes = plt.subplots(rows, 3, figsize=(15, 5*rows))

    for i, pdb_chain in enumerate(unique_pdb_chains):
        ax = axes[i // 3, i % 3]
        sns.scatterplot(x='network_score', y='mutagenesis_functional_score', data=residue_data[residue_data['pdb_chain']==pdb_chain], ax=ax)
        ax.set_xlabel('Network Score')
        ax.set_ylabel('Mutagenesis Functional Score')
        ax.set_title(f'{pdb_chain}')
        ax.grid(True)

    # set supertitle at the top of the figure
    plt.suptitle(f"Network Score vs Average Mutagenesis Functional Score (Z-score) for {gene}", y=1.1, verticalalignment='top', fontsize = 15)

    # Remove any unused subplots
    for j in range(i+1, rows*3):
        fig.delaxes(axes.flatten()[j])

    plt.tight_layout()

    # Save the plots in the PDF document
    pdf_pages.savefig(fig)

    # # Close the PdfPages object
    pdf_pages.close()

    plt.close()
    # plt.show()

## 2. Interactive dash

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd

df = residue_data

# Get unique genes
unique_genes = df['gene_symbol'].unique()

# Initialize Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    dcc.Dropdown(
        id='gene-dropdown',
        options=[{'label': gene, 'value': gene} for gene in unique_genes],
        value='TP53',  # Default value set to 'TP53'
        clearable=False
    ),
    html.Br(),
    dcc.Dropdown(
        id='pdb-dropdown',
        options=[{'label': 'All PDBs', 'value': 'all'}] + [{'label': pdb_id, 'value': pdb_id} for pdb_id in df['pdb_chain'].unique()],
        multi=True
    ),
    dcc.Graph(id='scatter-plot')
])

# Define callback to update PDB dropdown based on selected gene
@app.callback(
    Output('pdb-dropdown', 'value'),
    [Input('gene-dropdown', 'value')]
)
def update_pdb_value(selected_gene):
    return ['all'] + list(df[df['gene_symbol'] == selected_gene]['pdb_chain'].unique())

# Define callback to update scatter plot
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('gene-dropdown', 'value'),
     Input('pdb-dropdown', 'value')]
)
def update_plot(selected_gene, selected_pdb_ids):

    if 'all' in selected_pdb_ids:
        filtered_df = df[df['gene_symbol'] == selected_gene]
    else:
        filtered_df = df[df['pdb_chain'].isin(selected_pdb_ids)]
    title = "Network score distribution against mutagenesis functional score"
    fig = px.scatter(filtered_df, x='network_score', y='mutagenesis_functional_score',
                     hover_data=['pdb_id', 'chain', 'res_num', 'pdb_res', 'uniprot_res'],
                     labels={'mutagenesis_functional_score': 'Functional Score from saturation mutagenesis',
                             'network_score': 'Network Score',
                             'residue_match': 'Residue Match',
                                    'pdb_id': 'PDB ID',
                                    'chain': 'Chain',
                                    'res_num': 'Residue Number',
                                    'pdb_res': 'PDB Residue',
                                    'uniprot_res': 'UniProt Residue'},
                     title=f'{title} ({len(selected_pdb_ids)} PDB chains selected)' if 'all' not in selected_pdb_ids else f'{title} (All PDB chains selected)')
    
    fig.update_xaxes(title_text='Network Score')
    fig.update_yaxes(title_text='Mutagenesis Functional Score')
    fig.update_traces(opacity=.4)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
